In [1]:
%matplotlib
import aotools
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
from scipy.integrate import trapz
plt.rcParams['image.interpolation'] = 'nearest'

Using matplotlib backend: TkAgg


In [5]:
## Generate sample data
diameter = 128
nzernike = 36
zcoeffs_in = 256 * np.random.rand(nzernike)# - 128
img = aotools.phaseFromZernikes(zcoeffs_in, diameter)

## Decomposition by double integral
zcoeffs_dbl = []
for i in range(1,nzernike+1):
    intermediate = trapz(img * aotools.zernike(i, diameter))
    zcoeffs_dbl.append(trapz(intermediate) / (diameter*diameter))
    
## Decomposition by cross-correlation
# Works only for +ve images.
imgfft = np.fft.fft2(img)
zcoeffs_fft = []
zs = []
for i in range(1,nzernike+1):
    zfft = np.fft.fft2(aotools.zernike(i, diameter))
    z = np.mean(imgfft * zfft) / (diameter*diameter)
    zcoeffs_fft.append(np.abs(z))
    
## Compare results
plt.figure()
gs = gridspec.GridSpec(2, 3)

ax = plt.subplot(gs[0, 0])
ax.imshow(img)
ax.set_title('input')

ax = plt.subplot(gs[0, 1])
ax.imshow(aotools.phaseFromZernikes(zcoeffs_dbl, diameter))
ax.set_title('double integral')

ax = plt.subplot(gs[0, 2])
ax.imshow(aotools.phaseFromZernikes(zcoeffs_fft, diameter))
ax.set_title('fft cross corr.')

ax = plt.subplot(gs[1,:])
ax.plot(range(nzernike), zcoeffs_in, 'b-', label="input")
ax.plot(range(nzernike), zcoeffs_dbl, 'r-', label="double integral")
ax.plot(range(nzernike), zcoeffs_fft, 'c-', label="fft cross-corr.")
ax.legend(loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.1))


In [ ]:
col = 'bgrcmyk'
diameter = 128
nzernike = 37
zcoeffs_in = np.random.rand(nzernike)
img = aotools.phaseFromZernikes(zcoeffs_in, diameter)

zc = []
zc_out = []
for m in range(5):
    zc.append(np.array(zcoeffs_in) + 0.05*m)
    img = aotools.phaseFromZernikes(np.array(zc[m]), diameter)
    imgfft = np.fft.fft2(img)
    
    zc_out.append([])
    for i in range(1,nzernike+1):
        zfft = np.fft.fft2(aotools.zernike(i, diameter))
        zc_out[m].append(np.abs(np.mean(imgfft * zfft)) / (diameter*diameter))
    
    c = col[m%len(col)]
    plt.plot(range(nzernike), zc[m], c)
    plt.plot(range(nzernike), zc_out[m], c+':')
plt.show()